# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab = set(text)
    
    vocab_to_int = {word: i for i,word in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))    
    
    print(vocab_to_int)
    print("\n")
    print(int_to_vocab)
    
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

{'meet': 0, 'i': 1, "moe's": 2, "i'll": 3, 'one': 4, 'tavern': 5, 'where': 6, 'hello': 7, 'enhance': 8, 'name': 9, 'the': 10, 'homer_simpson': 11, 'is': 12, 'puke': 13, 'your': 14, 'seen': 15, 'anybody': 16, 'of': 17, 'and': 18, 'got': 19, 'effervescent': 20, 'whats': 21, 'should': 22, 'moe_szyslak': 23, 'another': 24, 'self': 25, 'to': 26, 'eh': 27, 'yeah': 28, 'not': 29, 'pick': 30, 'rotch': 31, 'with': 32, 'social': 33, 'bart_simpson': 34, 'catch': 35, 'has': 36, 'my': 37, 'normal': 38, 'only': 39, 'lately': 40, "you're": 41, 'give': 42, 'homer': 43, 'hey': 44, 'mike': 45, 'barney_gumble': 46, 'there': 47, 'these': 48, 'an': 49, 'back': 50, 'ice': 51, 'check': 52, 'drink': 53, 'you': 54, 'little': 55, 'matter': 56, 'moe': 57, 'on': 58, 'problems': 59, 'elite': 60, 'carve': 61, 'gonna': 62, 'listen': 63, 'forget': 64, 'skills': 65, 'last': 66, 'hold': 67, "i'm": 68, 'days': 69, 'me': 70}


{0: 'meet', 1: 'i', 2: "moe's", 3: "i'll", 4: 'one', 5: 'tavern', 6: 'where', 7: 'hello', 8: 'e

### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    punc = {}
    punc["."] = "Period"
    punc[","] = "Comma"
    punc["\""] = "Quotation_Mark"
    punc[";"] = "Semicolon"
    punc["!"] = "Exclamation_mark"
    punc["?"] = "Question_mark"
    punc["("] = "Left_Parentheses"
    punc[")"] = "Right_Parentheses"
    punc["--"] = "Dash"
    punc["\n"] = "Return"
    
    return punc

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

{'during': 0, 'hampstead-on-cecil-cecil': 1, 'trying': 2, 'crime': 3, 'lecture': 4, 'marvelous': 5, 'appear': 6, 'umm': 7, 'supreme': 8, 'duffman:': 9, 'talkers': 10, 'composite': 11, 'cruiser': 12, 'alec_baldwin:': 13, 'gardens': 14, 'safer': 15, 'hah': 16, 'moment': 17, 'museum': 18, 'smell': 19, 'grab': 20, 'mystery': 21, 'popping': 22, 'slaves': 23, 'slogan': 24, 'undermine': 25, 'fry': 26, 'peeping': 27, 'amazing': 28, "gettin'": 29, 'jets': 30, 'allowed': 31, 'nickels': 32, 'idiots': 33, 'tied': 34, 'aer': 35, 'pope': 36, 'changed': 37, 'truck': 38, 'bye': 39, 'i': 40, 'comic_book_guy:': 41, 'cough': 42, 'tree': 43, 'jerks': 44, 'chorus:': 45, 'cheery': 46, 'micronesian': 47, 'somebody': 48, 'não': 49, 'pissed': 50, 'subscriptions': 51, 'kl5-4796': 52, 'wings': 53, 'life-partner': 54, 'embarrassing': 55, 'grocery': 56, 'heavens': 57, 'cesss': 58, 'together': 59, 'closer': 60, 'empty': 61, 'wagering': 62, 'dr': 63, 'stealings': 64, 'husband': 65, 'turkey': 66, 'trouble': 67, 'dict

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [28]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, [None, None], name="input")
    targets = tf.placeholder(tf.int32, [None, None], name="targets")
    learning_rate = tf.placeholder(tf.float32, name="learning_rate")
    
    print("inputs shape is {}" .format(inputs.shape))
    print("targets shape is {}" .format(targets.shape))
    
    return inputs, targets, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

inputs shape is (?, ?)
targets shape is (?, ?)
Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [29]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([lstm])
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, "initial_state")
    
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [30]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.truncated_normal((vocab_size, embed_dim)))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [31]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, "final_state")
    
    print("inputs shape is {}" .format(inputs.shape))
    print("outputs shape is {}" .format(outputs.shape))
    print("final_state shape is {}" .format(final_state.shape))
    
    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

inputs shape is (?, ?, 256)
outputs shape is (?, ?, 256)
final_state shape is (2, 2, ?, 256)
Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [32]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embed = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embed)        
    
    #flattened = tf.contrib.layers.flatten(outputs)
    #out = tf.reshape(flattened, [-1, rnn_size])
    #print("outputs shape after reshaping is {}" .format(out.shape))
    
    # softmax layer     
    #softmax_w = tf.Variable(tf.truncated_normal([rnn_size, vocab_size]))
    #softmax_b = tf.Variable(tf.zeros(vocab_size))
    
    #logits = tf.add(tf.matmul(out, softmax_w), softmax_b)
    #logits = tf.reshape(logits, [outputs.shape[0], outputs.shape[1], -1])    
    
    '''
    what is the difference between the above implementation and using :
    '''
    
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)    
    print("logits shape is {}" .format(logits.shape))
    
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

inputs shape is (128, 5, 300)
outputs shape is (128, 5, 256)
final_state shape is (2, 2, 128, 256)
logits shape is (128, 5, 27)
Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [33]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    chars_per_batch = batch_size * seq_length
    num_of_batches = len(int_text) // chars_per_batch
    
    inputs = np.array(int_text[:num_of_batches*chars_per_batch])        
    targets = np.array(int_text[1:num_of_batches*chars_per_batch] + [int_text[0]])
    
    print("inputs shape = ", inputs.shape)
    print("targets shape = ", targets.shape)
    
    inputs = inputs.reshape(batch_size, -1)
    targets = targets.reshape(batch_size, -1)
    
    inputs = np.split(inputs, num_of_batches, axis=1)
    targets = np.split(targets, num_of_batches, axis=1)
    
    batches = np.array(list(zip(inputs, targets)))
    batches.reshape(num_of_batches, 2, batch_size, seq_length)
    
    return batches


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

inputs shape =  (4480,)
targets shape =  (4480,)
Tests Passed


In [34]:
batches = get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25], 3, 2)
print(batches)

inputs shape =  (24,)
targets shape =  (24,)
[[[[ 1  2]
   [ 9 10]
   [17 18]]

  [[ 2  3]
   [10 11]
   [18 19]]]


 [[[ 3  4]
   [11 12]
   [19 20]]

  [[ 4  5]
   [12 13]
   [20 21]]]


 [[[ 5  6]
   [13 14]
   [21 22]]

  [[ 6  7]
   [14 15]
   [22 23]]]


 [[[ 7  8]
   [15 16]
   [23 24]]

  [[ 8  9]
   [16 17]
   [24  1]]]]


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [35]:
# Number of Epochs
num_epochs = 50
# Batch Size
batch_size = 64
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 300
# Sequence Length
seq_length = 16
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 15

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [36]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

inputs shape is (?, ?)
targets shape is (?, ?)
inputs shape is (?, ?, 300)
outputs shape is (?, ?, 256)
final_state shape is (1, 2, ?, 256)
logits shape is (?, ?, 6776)


## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [37]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

inputs shape =  (68608,)
targets shape =  (68608,)
Epoch   0 Batch    0/67   train_loss = 8.820
Epoch   0 Batch   15/67   train_loss = 6.738
Epoch   0 Batch   30/67   train_loss = 6.324
Epoch   0 Batch   45/67   train_loss = 6.217
Epoch   0 Batch   60/67   train_loss = 5.924
Epoch   1 Batch    8/67   train_loss = 5.608
Epoch   1 Batch   23/67   train_loss = 5.667
Epoch   1 Batch   38/67   train_loss = 5.730
Epoch   1 Batch   53/67   train_loss = 5.728
Epoch   2 Batch    1/67   train_loss = 5.320
Epoch   2 Batch   16/67   train_loss = 5.358
Epoch   2 Batch   31/67   train_loss = 5.175
Epoch   2 Batch   46/67   train_loss = 5.207
Epoch   2 Batch   61/67   train_loss = 5.199
Epoch   3 Batch    9/67   train_loss = 5.001
Epoch   3 Batch   24/67   train_loss = 4.998
Epoch   3 Batch   39/67   train_loss = 4.902
Epoch   3 Batch   54/67   train_loss = 5.046
Epoch   4 Batch    2/67   train_loss = 4.685
Epoch   4 Batch   17/67   train_loss = 4.973
Epoch   4 Batch   32/67   train_loss = 4.949
Epoc

Epoch  40 Batch   35/67   train_loss = 2.167
Epoch  40 Batch   50/67   train_loss = 2.128
Epoch  40 Batch   65/67   train_loss = 2.129
Epoch  41 Batch   13/67   train_loss = 2.074
Epoch  41 Batch   28/67   train_loss = 2.087
Epoch  41 Batch   43/67   train_loss = 2.109
Epoch  41 Batch   58/67   train_loss = 2.075
Epoch  42 Batch    6/67   train_loss = 2.024
Epoch  42 Batch   21/67   train_loss = 2.163
Epoch  42 Batch   36/67   train_loss = 2.038
Epoch  42 Batch   51/67   train_loss = 2.087
Epoch  42 Batch   66/67   train_loss = 1.969
Epoch  43 Batch   14/67   train_loss = 2.111
Epoch  43 Batch   29/67   train_loss = 2.118
Epoch  43 Batch   44/67   train_loss = 1.956
Epoch  43 Batch   59/67   train_loss = 2.041
Epoch  44 Batch    7/67   train_loss = 2.060
Epoch  44 Batch   22/67   train_loss = 2.074
Epoch  44 Batch   37/67   train_loss = 1.976
Epoch  44 Batch   52/67   train_loss = 2.088
Epoch  45 Batch    0/67   train_loss = 2.085
Epoch  45 Batch   15/67   train_loss = 1.914
Epoch  45 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [38]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [39]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [40]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [41]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    highest_prop_idx = np.argmax(probabilities)
    predicted_word = int_to_vocab[highest_prop_idx]
    
    return predicted_word


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [42]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak:(victorious chuckle) no, i can't believe that one pin all right.
moe_szyslak:(to homer) i know, i know, i don't know. this is the kitchen, this is the busiest drinkin' day of the world, they put the complaint department...
homer_simpson:(gasp) ooh, moe, moe.
moe_szyslak:(to homer) i know, i just wanted to talk, but i want you to see the next fifteen minutes, then add sloe gin, but then.
moe_szyslak:(to homer) i guess...
moe_szyslak:(to homer) i know, i just talk to me.
moe_szyslak:(excited) oh, i'm sorry, moe. you know, i just wanted to be a lot happier...
moe_szyslak:(to homer) i know, i just talk to me.
moe_szyslak:(excited) oh, i'm sorry, moe. you know, i just wanted to be a lot happier...
moe_szyslak:(to homer) i know


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.